# 5. 협업 필터링 알고리즘

이 챕터에서는 협업 필터링의 두 가지 주요 접근법인 사용자 기반 협업 필터링(User-Based Collaborative Filtering)과 아이템 기반 협업 필터링(Item-Based Collaborative Filtering)을 구현합니다. 이러한 메모리 기반 방법은 사용자 또는 아이템 간의 유사성을 계산하여 추천을 생성합니다.

## 5.1 필요한 라이브러리 불러오기

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from collections import defaultdict
from typing import Dict, List
from sklearn.metrics.pairwise import cosine_similarity

# 상위 디렉토리 경로를 시스템 경로에 추가하여 utils 모듈을 import할 수 있게 함
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# utils 모듈에서 필요한 클래스와 함수를 import
from utils.models import Dataset, RecommendResult, Metrics
from utils.data_loader import DataLoader

## 5.2 추천 점수 정렬 유틸리티 함수

In [ ]:
def _recommend_sorted(scores: Dict[int,float], seen: set, k: int) -> List[int]:
    """예측 점수를 기반으로 상위 k개 아이템을 추천 목록으로 생성하는 유틸리티 함수

    1. 사용자가 아직 평가하지 않은 아이템 중에서 (seen에 없는 아이템)
    2. 예측 점수가 높은 순서로 정렬하여 상위 k개 선택 (동일 점수는 아이템 ID 오름차순)

    매개변수:
    - scores: {아이템 ID: 예측 점수} 형태의 딕셔너리
    - seen: 사용자가 이미 평가한 아이템 ID 집합
    - k: 추천할 아이템 개수

    반환값:
    - 추천할 상위 k개 아이템 ID 리스트
    """
    # (-score, movieId asc) 형태로 정렬
    cands = ((m, s) for m, s in scores.items() if m not in seen)
    return [m for m,_ in sorted(cands, key=lambda x: (-x[1], x[0]))[:k]]

## 5.3 사용자 기반 협업 필터링 (UserKNN)

In [ ]:
class UserKNNRecommender(BaseRecommender):
    """사용자 기반 협업 필터링 (User-based Collaborative Filtering)

    유사한 사용자들이 좋아한 아이템을 추천하는 방식입니다.
    사용자-아이템 평점 행렬에서 사용자 간 유사도를 계산하고,
    타겟 사용자와 유사한 사용자들의 평점을 가중평균하여 예측합니다.

    알고리즘 로직:
    1. 사용자-아이템 행렬 생성:
       - 행: 사용자 ID, 열: 영화 ID, 값: 평점
       - 평가하지 않은 항목은 0으로 채움

    2. 사용자 간 코사인 유사도 계산:
       - 각 사용자 벡터 간의 코사인 유사도를 계산
       - 유사도가 높을수록 취향이 비슷함을 의미

    3. 평점 예측 (RMSE 계산용):
       - 각 테스트 아이템에 대해 대상 사용자와 유사한 n_neighbors명의 이웃을 선택
       - 이웃 중 해당 아이템을 평가한 사용자의 평점을 유사도로 가중평균하여 예측
       - 이웃 중 아무도 해당 아이템을 평가하지 않았다면 전체 평균(μ) 사용
       - 예측값을 0.5~5.0 범위로 제한

    4. 아이템 추천:
       - 각 사용자가 아직 평가하지 않은 모든 아이템에 대해 예측 평점을 계산
       - 예측 평점이 높은 순으로 상위 k개 아이템을 추천

    매개변수:
    - k: 추천 목록 크기 (기본값: 10)
    - n_neighbors: 유사도 계산에 사용할 이웃 수 (기본값: 20)
    """
    def recommend(self, dataset: Dataset, k=10, n_neighbors=20, **kwargs) -> RecommendResult:
        train, test = dataset.train, dataset.test
        mu = float(train['rating'].mean())  # 전체 평점 평균

        # 사용자-아이템 행렬 생성 (ui: User-Item matrix)
        ui = train.pivot_table(index='userId', columns='movieId', values='rating').fillna(0.0)
        # 사용자 간 코사인 유사도 계산
        sim = cosine_similarity(ui)
        sim_df = pd.DataFrame(sim, index=ui.index, columns=ui.index)
        # 사용자별 이미 평가한 아이템 목록
        seen = train.groupby('userId')['movieId'].apply(set).to_dict()

        # RMSE 계산을 위한 평점 예측
        pred = pd.Series(index=test.index, dtype=float)
        for idx, row in test.iterrows():
            u, i = row['userId'], row['movieId']  # 사용자 ID와 영화 ID
            # 학습 데이터에 없는 사용자나 아이템은 전체 평균 사용
            if (u not in ui.index) or (i not in ui.columns):
                pred.at[idx] = mu; continue
            # 자신을 제외한 가장 유사한 n_neighbors명의 이웃 선택
            neigh = sim_df.loc[u].sort_values(ascending=False).iloc[1:n_neighbors+1]
            # 선택된 이웃들의 해당 영화 평점
            neigh_r = ui.loc[neigh.index, i]
            # 해당 영화를 평가한 이웃만 선택 (평점 > 0)
            mask = neigh_r > 0
            if mask.sum()==0: pred.at[idx]=mu  # 평가한 이웃이 없으면 전체 평균 사용
            else:
                # 유사도를 가중치로 하여 평점 가중평균 계산
                w = neigh[mask]; r = neigh_r[mask]; s=w.sum()
                pred.at[idx] = float((w*r).sum()/s) if s>1e-8 else mu
        # 예측값 범위 조정 (0.5~5.0)
        pred = pred.clip(lower=0.5, upper=5.0)

        # 추천 목록 생성
        user2items = defaultdict(list)
        for u in test['userId'].unique():
            # 학습 데이터에 없는 사용자는 모든 영화에 평균 점수 부여
            if u not in ui.index:
                scores = {m: mu for m in ui.columns}
            else:
                # 아직 평가하지 않은 영화 목록
                unseen = [m for m in ui.columns if m not in seen.get(u,set())]
                # 가장 유사한 이웃 선택
                neigh = sim_df.loc[u].sort_values(ascending=False).iloc[1:n_neighbors+1]
                scores = {}
                # 각 영화마다 예측 점수 계산
                for i in unseen:
                    neigh_r = ui.loc[neigh.index, i]
                    mask = neigh_r > 0
                    if mask.sum()==0: scores[i]=mu  # 평가한 이웃이 없으면 전체 평균 사용
                    else:
                        # 유사도 가중 평점 계산
                        w=neigh[mask]; r=neigh_r[mask]; s=w.sum()
                        scores[i]=float((w*r).sum()/s) if s>1e-8 else mu
            # 점수가 높은 순서로 상위 k개 영화 추천
            user2items[u] = _recommend_sorted(scores, seen.get(u,set()), k)
        return RecommendResult(pred, user2items)

## 5.4 아이템 기반 협업 필터링 (ItemKNN)

In [ ]:
class ItemKNNRecommender(BaseRecommender):
    """아이템 기반 협업 필터링 (Item-based Collaborative Filtering)

    사용자가 평가한 아이템과 유사한 다른 아이템을 추천하는 방식입니다.
    아이템 간 유사도를 계산하고, 사용자가 높이 평가한 아이템과 유사한 아이템을 추천합니다.

    알고리즘 로직:
    1. 사용자-아이템 행렬 생성 및 전치:
       - 원본: 행=사용자, 열=영화
       - 전치(iu): 행=영화, 열=사용자 (Item-User matrix)

    2. 아이템 간 코사인 유사도 계산:
       - 각 영화 벡터 간의 코사인 유사도를 계산
       - 유사도가 높을수록 비슷한 영화임을 의미

    3. 평점 예측 (RMSE 계산용):
       - 예측할 영화와 가장 유사한 n_neighbors개의 영화를 선택
       - 그 중 사용자가 평가한 영화들만 고려(common)
       - 유사도를 가중치로 하여 사용자의 평점을 가중평균
       - 유사한 영화를 사용자가 평가하지 않았다면 전체 평균(μ) 사용
       - 예측값을 0.5~5.0 범위로 제한

    4. 아이템 추천:
       - 각 사용자가 아직 평가하지 않은 모든 영화에 대해 예측 평점 계산
       - 예측 평점이 높은 순으로 상위 k개 영화를 추천

    매개변수:
    - k: 추천 목록 크기 (기본값: 10)
    - n_neighbors: 유사도 계산에 사용할 이웃 수 (기본값: 20)

    특징:
    - 사용자 기반보다 일반적으로 더 안정적인 성능을 보임
    - 아이템 간 유사도는 변화가 적어 미리 계산해두면 효율적
    - Cold-start 문제에 비교적 강건함
    """
    def recommend(self, dataset: Dataset, k=10, n_neighbors=20, **kwargs) -> RecommendResult:
        train, test = dataset.train, dataset.test
        mu = float(train['rating'].mean())  # 전체 평점 평균

        # 사용자-아이템 행렬 생성 및 전치하여 아이템-사용자 행렬로 변환
        ui = train.pivot_table(index='userId', columns='movieId', values='rating').fillna(0.0)
        iu = ui.T  # 전치: 행=영화, 열=사용자
        # 아이템 간 코사인 유사도 계산
        sim = cosine_similarity(iu)
        sim_df = pd.DataFrame(sim, index=iu.index, columns=iu.index)
        # 사용자별 이미 평가한 아이템 목록
        seen = train.groupby('userId')['movieId'].apply(set).to_dict()

        # RMSE 계산을 위한 평점 예측
        pred = pd.Series(index=test.index, dtype=float)
        for idx, row in test.iterrows():
            u, i = row['userId'], row['movieId']  # 사용자 ID와 영화 ID
            # 학습 데이터에 없는 사용자나 아이템은 전체 평균 사용
            if (u not in ui.index) or (i not in ui.columns):
                pred.at[idx] = mu; continue
            # 현재 영화와 가장 유사한 n_neighbors개의 영화 선택
            sims = sim_df[i].sort_values(ascending=False).iloc[1:n_neighbors+1]
            # 현재 사용자가 평가한 영화들
            rated = ui.loc[u]
            # 유사한 영화 중 사용자가 평가한 영화만 선택
            common = sims.index[rated.loc[sims.index] > 0]
            if len(common)==0: pred.at[idx]=mu  # 공통 영화가 없으면 전체 평균 사용
            else:
                # 유사도를 가중치로 하여 평점 가중평균 계산
                w=sims.loc[common]; r=rated.loc[common]; s=w.sum()
                pred.at[idx]=float((w*r).sum()/s) if s>1e-8 else mu
        # 예측값 범위 조정 (0.5~5.0)
        pred = pred.clip(lower=0.5, upper=5.0)

        # 추천 목록 생성
        user2items = defaultdict(list)
        for u in test['userId'].unique():
            # 학습 데이터에 없는 사용자는 모든 영화에 평균 점수 부여
            if u not in ui.index:
                scores = {m: mu for m in ui.columns}
            else:
                # 아직 평가하지 않은 영화 목록
                unseen = [m for m in ui.columns if m not in seen.get(u,set())]
                # 해당 사용자가 평가한 영화들
                rated = ui.loc[u]
                scores = {}
                # 각 미평가 영화에 대해 예측 점수 계산
                for i in unseen:
                    # 현재 영화와 가장 유사한 n_neighbors개의 영화 선택
                    sims = sim_df[i].sort_values(ascending=False).iloc[1:n_neighbors+1]
                    # 유사한 영화 중 사용자가 평가한 영화만 선택
                    common = sims.index[rated.loc[sims.index] > 0]
                    if len(common)==0: scores[i]=mu  # 공통 영화가 없으면 전체 평균 사용
                    else:
                        # 유사도 가중 평점 계산
                        w=sims.loc[common]; r=rated.loc[common]; s=w.sum()
                        scores[i]=float((w*r).sum()/s) if s>1e-8 else mu
            # 점수가 높은 순서로 상위 k개 영화 추천
            user2items[u] = _recommend_sorted(scores, seen.get(u,set()), k)
        return RecommendResult(pred, user2items)

## 5.5 협업 필터링 알고리즘 평가

In [ ]:
# 데이터셋 로드
loader = DataLoader(num_users=100)
dataset = loader.load()

# 메트릭스 계산기
mc = MetricCalculator()
K = 10  # 추천 목록 크기

# UserKNN 평가 (다양한 이웃 수로 실험)
print("\n=== UserKNN 평가 ===")
uk_results = []
for n in [5, 10, 20]:
    uk_recommender = UserKNNRecommender()
    uk_result = uk_recommender.recommend(dataset, k=K, n_neighbors=n)
    metrics = mc.calc(
        dataset.test['rating'].tolist(),
        uk_result.rating.tolist(),
        dataset.test_user2items,
        uk_result.user2items,
        k=K,
        params={"model": "UserKNN", "n_neighbors": n}
    )
    uk_results.append(metrics)
    print(f"UserKNN (n_neighbors={n}):", metrics)

# 최고 성능의 UserKNN 선택
best_uk = max(uk_results, key=lambda x: (x.precision_at_k, -x.rmse))
print("Best UserKNN:", best_uk)

# ItemKNN 평가 (다양한 이웃 수로 실험)
print("\n=== ItemKNN 평가 ===")
ik_results = []
for n in [5, 10, 20]:
    ik_recommender = ItemKNNRecommender()
    ik_result = ik_recommender.recommend(dataset, k=K, n_neighbors=n)
    metrics = mc.calc(
        dataset.test['rating'].tolist(),
        ik_result.rating.tolist(),
        dataset.test_user2items,
        ik_result.user2items,
        k=K,
        params={"model": "ItemKNN", "n_neighbors": n}
    )
    ik_results.append(metrics)
    print(f"ItemKNN (n_neighbors={n}):", metrics)

# 최고 성능의 ItemKNN 선택
best_ik = max(ik_results, key=lambda x: (x.precision_at_k, -x.rmse))
print("Best ItemKNN:", best_ik)

## 5.6 협업 필터링 알고리즘 비교 및 분석

이 챕터에서 구현한 두 가지 협업 필터링 접근법의 비교입니다:

### 사용자 기반 협업 필터링 (UserKNN)
- **장점**:
  - 직관적이고 구현이 쉬움
  - 새로운 아이템이 추가될 때 즉시 추천 가능 (cold-start 문제 없음)
  - 사용자의 취향 변화에 민감하게 반응
- **단점**:
  - 사용자 수가 많을 때 계산 비용이 높음
  - 희소성 문제에 취약함 (평가 데이터가 적은 경우)
  - 사용자 행동이 자주 변경되면 유사도도 자주 재계산 필요

### 아이템 기반 협업 필터링 (ItemKNN)
- **장점**:
  - 아이템 관계는 안정적이므로 유사도 행렬을 미리 계산해 둘 수 있음
  - 일반적으로 사용자 기반보다 더 나은 확장성
  - Cold-start 문제에 더 강건함
  - 추천 결과에 대한 설명이 용이함 ("당신이 좋아한 X와 유사한 Y를 추천합니다")
- **단점**:
  - 새 아이템 추가 시 유사도 재계산 필요
  - 아이템이 많을 경우 메모리 요구량이 증가
  
### 성능 최적화 팁
- **희소 행렬(Sparse Matrix) 사용**: 대규모 데이터에서는 희소 행렬(예: CSR 행렬)을 사용하여 메모리 효율성 향상
- **이웃 수 최적화**: n_neighbors 값은 성능에 큰 영향을 미치므로 그리드 서치로 최적값 찾기
- **피어슨 상관계수 시도**: 코사인 유사도 대신 피어슨 상관계수를 사용하여 유사도 측정 성능 비교
- **평점 정규화**: 사용자별 평점 패턴 차이를 보정하기 위한 평점 정규화(중심화) 적용

다음 챕터에서는 행렬 분해(Matrix Factorization) 기법을 통해 모델 기반 접근법을 살펴보겠습니다.